In [33]:
%pip install pandas numpy faker

Note: you may need to restart the kernel to use updated packages.


# Dữ Liệu Giao Dịch E-commerce/Gaming Việt Nam

## Phần 1: Xây dựng cấu trúc và giả lập dữ liệu gốc (1000 giao dịch)

In [34]:
import pandas as pd
import numpy as np
import json
from datetime import datetime, timedelta
import random
from faker import Faker
import uuid

# Cấu hình Faker cho Việt Nam
fake = Faker('vi_VN')
np.random.seed(42)
random.seed(42)

print("Thư viện đã được import thành công!")

Thư viện đã được import thành công!


In [35]:
# Định nghĩa các cấu trúc dữ liệu cho thị trường Việt Nam
customer_segments = {
    'Tân_Binh': {'weight': 0.25, 'avg_spend': 50000, 'description': 'Khách hàng mới, chi tiêu thấp'},
    'Tiem_Nang': {'weight': 0.20, 'avg_spend': 150000, 'description': 'Có tiềm năng phát triển'},
    'Thuong_Thien': {'weight': 0.30, 'avg_spend': 350000, 'description': 'Khách hàng trung thành'},
    'VIP': {'weight': 0.15, 'avg_spend': 800000, 'description': 'Khách hàng cao cấp'},
    'Doanh_Nghiep': {'weight': 0.10, 'avg_spend': 2000000, 'description': 'Khách hàng doanh nghiệp'}
}

transaction_types = [
    'Mua Game', 'Nạp Tiền Game', 'Mua Vật Phẩm', 'Thanh Toán Premium', 
    'Đăng Ký Gói Cước', 'Mua Ứng Dụng', 'Đăng Ký Tài Khoản Premium',
    'Nạp Thẻ Điện Thoại', 'Mua Víp Game', 'Thanh Toán Dịch Vụ'
]

download_channels = [
    'App Store', 'Google Play', 'Website Official', 
    'CH Play', 'App Gallery', 'Direct Download',
    'Steam', 'Epic Games', 'Garena', 'VNG Game'
]

payment_methods = [
    'Ví Momo', 'Ví ZaloPay', 'Thẻ ATM', 'Thẻ Tín Dụng',
    'Tiền Mặt', 'Chuyển Khoản', 'Ví ShopeePay', 'Ví VNPay'
]

cities_vietnam = [
    'Hà Nội', 'TP Hồ Chí Minh', 'Đà Nẵng', 'Hải Phòng', 'Cần Thơ',
    'Buôn Ma Thuột', 'Huế', 'Nha Trang', 'Vũng Tàu', 'Biên Hòa',
    'Thủ Đức', 'Long An', 'Đồng Nai', 'Bình Dương', 'Thái Nguyên'
]

device_types = ['Android', 'iOS', 'Windows', 'macOS', 'Web']

print("Cấu trúc dữ liệu đã được định nghĩa!")

Cấu trúc dữ liệu đã được định nghĩa!


In [36]:
# Hàm tạo ID khách hàng theo định dạng Việt Nam
def generate_customer_id():
    return f"KH{random.randint(100000, 999999)}"

# Hàm tạo mã giao dịch
def generate_transaction_id():
    timestamp = datetime.now().strftime('%Y%m%d')
    random_str = ''.join(random.choices('ABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789', k=6))
    return f"TXN{timestamp}{random_str}"

# Hàm chọn theo trọng số
def weighted_choice(items, weights):
    return random.choices(items, weights=weights)[0]

# Hàm tạo số điện thoại Việt Nam
def generate_vietnam_phone():
    prefixes = ['09', '08', '03', '07', '05']
    prefix = random.choice(prefixes)
    suffix = ''.join(random.choices('0123456789', k=8))
    return f"{prefix}{suffix}"

print("Hàm tiện ích đã được tạo!")

Hàm tiện ích đã được tạo!


In [37]:
# Tạo dữ liệu gốc 1000 giao dịch
def generate_transaction_data(num_transactions=1000):
    
    transactions = []
    customers = {}
    
    # Tạo khoảng thời gian 3 tháng qua
    end_date = datetime.now()
    start_date = end_date - timedelta(days=90)
    
    for i in range(num_transactions):
        # Tạo hoặc lấy thông tin khách hàng
        if random.random() < 0.3:  # 30% là khách hàng mới
            customer_id = generate_customer_id()
            if customer_id not in customers:
                segment = weighted_choice(
                    list(customer_segments.keys()),
                    [seg['weight'] for seg in customer_segments.values()]
                )
                customers[customer_id] = {
                    'customer_id': customer_id,
                    'customer_name': fake.name(),
                    'phone': generate_vietnam_phone(),
                    'email': fake.email(),
                    'city': random.choice(cities_vietnam),
                    'segment': segment,
                    'registration_date': fake.date_between(start_date=start_date, end_date=end_date),
                    'device_type': random.choice(device_types)
                }
        else:
            customer_id = random.choice(list(customers.keys())) if customers else generate_customer_id()
            if customer_id not in customers:
                segment = 'Tân_Binh'
                customers[customer_id] = {
                    'customer_id': customer_id,
                    'customer_name': fake.name(),
                    'phone': generate_vietnam_phone(),
                    'email': fake.email(),
                    'city': random.choice(cities_vietnam),
                    'segment': segment,
                    'registration_date': fake.date_between(start_date=start_date, end_date=end_date),
                    'device_type': random.choice(device_types)
                }
        
        customer_info = customers[customer_id]
        
        # Tạo giao dịch
        segment_info = customer_segments[customer_info['segment']]
        base_amount = segment_info['avg_spend']
        
        # Thêm độ nhiễu vào số tiền
        amount_multiplier = np.random.normal(1.0, 0.3)
        amount = max(1000, int(base_amount * amount_multiplier))
        
        # Làm tròn số tiền theo mệnh giá phổ biến ở VN
        if amount < 10000:
            amount = (amount // 1000) * 1000
        else:
            amount = (amount // 10000) * 10000
        
        transaction = {
            'transaction_id': generate_transaction_id(),
            'customer_id': customer_id,
            'transaction_time': fake.date_time_between(start_date=start_date, end_date=end_date),
            'transaction_type': random.choice(transaction_types),
            'channel': random.choice(download_channels),
            'amount_vnd': amount,
            'payment_method': random.choice(payment_methods),
            'status': random.choice(['Thành Công', 'Thành Công', 'Thành Công', 'Thất Bại', 'Đang Xử Lý']),
            'ip_address': fake.ipv4(),
            'user_agent': fake.user_agent(),
            'promotion_code': random.choice(['', 'SALE50', 'WELCOME10', 'VIP20', '', 'FREESHIP', 'NEWUSER15']),
            'session_duration': random.randint(30, 3600),  # giây
            'items_count': random.randint(1, 5)
        }
        
        transactions.append(transaction)
    
    return transactions, list(customers.values())

# Tạo dữ liệu
print("Đang tạo 1000 giao dịch...")
transactions, customers = generate_transaction_data(1000)
print(f"Đã tạo {len(transactions)} giao dịch và {len(customers)} khách hàng!")

Đang tạo 1000 giao dịch...
Đã tạo 1000 giao dịch và 304 khách hàng!


In [38]:
# Chuyển thành DataFrame
df_transactions = pd.DataFrame(transactions)
df_customers = pd.DataFrame(customers)

# Sắp xếp theo thời gian
df_transactions = df_transactions.sort_values('transaction_time').reset_index(drop=True)

print("Kích thước dữ liệu giao dịch:", df_transactions.shape)
print("Kích thước dữ liệu khách hàng:", df_customers.shape)
print("\n5 giao dịch đầu tiên:")
df_transactions.head()

Kích thước dữ liệu giao dịch: (1000, 13)
Kích thước dữ liệu khách hàng: (304, 8)

5 giao dịch đầu tiên:


,transaction_id,customer_id,transaction_time,transaction_type,channel,amount_vnd,payment_method,status,ip_address,user_agent,promotion_code,session_duration,items_count
0,TXN20251022CAMBHW,KH940561,2025-07-24 17:09:39.078466,Mua Víp Game,Website Official,100000,Ví ZaloPay,Thành Công,8.244.247.221,Opera/8.82.(Windows NT 6.2; lo-LA) Presto/2.9....,WELCOME10,1194,5
1,TXN20251022CHU3YC,KH849259,2025-07-24 17:35:02.845311,Đăng Ký Tài Khoản Premium,App Gallery,1960000,Thẻ ATM,Thành Công,23.214.124.4,Mozilla/5.0 (compatible; MSIE 6.0; Windows NT ...,WELCOME10,2516,3
2,TXN20251022CFS7I1,KH882304,2025-07-24 18:28:33.795895,Mua Ứng Dụng,Direct Download,2220000,Ví ZaloPay,Thất Bại,73.20.44.162,Opera/8.54.(X11; Linux x86_64; mg-MG) Presto/2...,WELCOME10,3105,1
3,TXN20251022OM7ZTR,KH701518,2025-07-24 18:43:09.276431,Đăng Ký Gói Cước,Google Play,60000,Ví VNPay,Thành Công,77.201.174.125,Opera/9.32.(Windows NT 6.2; zu-ZA) Presto/2.9....,NEWUSER15,2758,3
4,TXN20251022CPKLV6,KH346205,2025-07-24 19:27:55.887065,Đăng Ký Gói Cước,Website Official,890000,Ví ShopeePay,Thất Bại,64.12.169.72,Opera/9.39.(Windows 95; gd-GB) Presto/2.9.189 ...,FREESHIP,531,1


In [ ]:
# Xuất dữ liệu gốc ra JSON
raw_data = {
    'transactions': df_transactions.to_dict('records'),
    'customers': df_customers.to_dict('records'),
    'metadata': {
        'total_transactions': len(df_transactions),
        'total_customers': len(df_customers),
        'date_range': {
            'start': df_transactions['transaction_time'].min().isoformat(),
            'end': df_transactions['transaction_time'].max().isoformat()
        },
        'generated_at': datetime.now().isoformat(),
        'description': 'Dữ liệu giao dịch e-commerce/gaming Việt Nam giả lập'
    }
}

# Lưu ra file JSON
with open('Dữ liệu gốc.json', 'w', encoding='utf-8') as f:
    json.dump(raw_data, f, ensure_ascii=False, indent=2, default=str)

print("Dữ liệu gốc đã được xuất ra file 'raw_transaction_data.json'")
print(f"Tổng cộng: {raw_data['metadata']['total_transactions']} giao dịch")
print(f"Khách hàng: {raw_data['metadata']['total_customers']} khách hàng")
print(f"Khoảng thời gian: {raw_data['metadata']['date_range']['start']} đến {raw_data['metadata']['date_range']['end']}")

# Hiển thị thông tin tổng quan
print("\nThống kê nhanh:")
print(f"Tổng doanh thu: {df_transactions['amount_vnd'].sum():,.0f} VNĐ")
print(f"Giá trị trung bình: {df_transactions['amount_vnd'].mean():,.0f} VNĐ")
print(f"Giao dịch thành công: {(df_transactions['status'] == 'Thành Công').sum()}")
print(f"Tỷ lệ thành công: {(df_transactions['status'] == 'Thành Công').mean()*100:.1f}%")

Dữ liệu gốc đã được xuất ra file 'raw_transaction_data.json'
Tổng cộng: 1000 giao dịch
Khách hàng: 304 khách hàng
Khoảng thời gian: 2025-07-24T17:09:39.078466 đến 2025-10-22T16:40:14.303560

Thống kê nhanh:
Tổng doanh thu: 498,009,000 VNĐ
Giá trị trung bình: 498,009 VNĐ
Giao dịch thành công: 597
Tỷ lệ thành công: 59.7%


## Phần 2: Xử lý và làm sạch dữ liệu

In [40]:
# Kiểm tra dữ liệu thiếu
print("Kiểm tra dữ liệu thiếu:")
print(df_transactions.isnull().sum())
print("\n" + "="*50 + "\n")

# Kiểm tra dữ liệu trùng lặp
print(f"Số giao dịch trùng lặp: {df_transactions.duplicated(subset=['transaction_id']).sum()}")
print("\n" + "="*50 + "\n")

# Kiểm tra tính hợp lý của dữ liệu
print("Kiểm tra số tiền âm:", (df_transactions['amount_vnd'] < 0).sum())
print("Giá trị giao dịch tối thiểu:", df_transactions['amount_vnd'].min())
print("Giá trị giao dịch tối đa:", df_transactions['amount_vnd'].max())
print("Giá trị giao dịch trung bình:", f"{df_transactions['amount_vnd'].mean():,.0f} VNĐ")

Kiểm tra dữ liệu thiếu:
transaction_id      0
customer_id         0
transaction_time    0
transaction_type    0
channel             0
amount_vnd          0
payment_method      0
status              0
ip_address          0
user_agent          0
promotion_code      0
session_duration    0
items_count         0
dtype: int64


Số giao dịch trùng lặp: 0


Kiểm tra số tiền âm: 0
Giá trị giao dịch tối thiểu: 9000
Giá trị giao dịch tối đa: 3840000
Giá trị giao dịch trung bình: 498,009 VNĐ


In [41]:
# Hàm làm sạch dữ liệu
def clean_transaction_data(df):
    df_clean = df.copy()
    
    # 1. Loại bỏ giao dịch trùng lặp
    before_count = len(df_clean)
    df_clean = df_clean.drop_duplicates(subset=['transaction_id'], keep='first')
    after_count = len(df_clean)
    print(f"Đã loại bỏ {before_count - after_count} giao dịch trùng lặp")
    
    # 2. Loại bỏ giá trị outlier (quá cao so với phân khúc)
    def is_amount_outlier(row):
        segment = row.get('segment', 'Tân_Binh')
        if segment in customer_segments:
            avg_amount = customer_segments[segment]['avg_spend']
            return row['amount_vnd'] > avg_amount * 5  # Ngưỡng 5 lần trung bình
        return False
    
    # Join với customer data để có segment
    df_with_segment = df_clean.merge(df_customers[['customer_id', 'segment']], 
                                   on='customer_id', how='left')
    
    outlier_mask = df_with_segment.apply(is_amount_outlier, axis=1)
    outliers_count = outlier_mask.sum()
    print(f"Phát hiện {outliers_count} giao dịch outlier")
    
    # Chuẩn hóa các outlier về giá trị hợp lý
    if outliers_count > 0:
        for idx in df_with_segment[outlier_mask].index:
            segment = df_with_segment.loc[idx, 'segment']
            if segment in customer_segments:
                avg_amount = customer_segments[segment]['avg_spend']
                df_clean.loc[idx, 'amount_vnd'] = int(avg_amount * random.uniform(2, 3))
        print(f"Đã chuẩn hóa {outliers_count} giao dịch outlier")
    
    # 3. Chuẩn hóa định dạng dữ liệu
    df_clean['transaction_time'] = pd.to_datetime(df_clean['transaction_time'])
    
    # 4. Thêm các trường tính toán
    df_clean['date'] = df_clean['transaction_time'].dt.date
    df_clean['hour'] = df_clean['transaction_time'].dt.hour
    df_clean['day_of_week'] = df_clean['transaction_time'].dt.day_name()
    df_clean['month'] = df_clean['transaction_time'].dt.to_period('M')
    
    # 5. Phân loại giao dịch theo khung thời gian
    def get_time_period(hour):
        if 6 <= hour < 12:
            return 'Sáng'
        elif 12 <= hour < 18:
            return 'Chiều'
        elif 18 <= hour < 22:
            return 'Tối'
        else:
            return 'Đêm'
    
    df_clean['time_period'] = df_clean['hour'].apply(get_time_period)
    
    # 6. Loại bỏ giao dịch có trạng thái không hợp lệ
    valid_statuses = ['Thành Công', 'Thất Bại', 'Đang Xử Lý']
    before_count = len(df_clean)
    df_clean = df_clean[df_clean['status'].isin(valid_statuses)]
    after_count = len(df_clean)
    if before_count != after_count:
        print(f"Đã loại bỏ {before_count - after_count} giao dịch có trạng thái không hợp lệ")
    
    # 7. Sắp xếp lại theo thời gian
    df_clean = df_clean.sort_values('transaction_time').reset_index(drop=True)
    
    return df_clean

# Áp dụng làm sạch dữ liệu
df_cleaned = clean_transaction_data(df_transactions)

print(f"\nSố lượng giao dịch sau khi làm sạch: {len(df_cleaned)}")
print(f"Giữ lại {(len(df_cleaned)/len(df_transactions)*100):.1f}% dữ liệu gốc")

Đã loại bỏ 0 giao dịch trùng lặp
Phát hiện 0 giao dịch outlier

Số lượng giao dịch sau khi làm sạch: 1000
Giữ lại 100.0% dữ liệu gốc


In [ ]:
# Xuất dữ liệu đã làm sạch ra JSON
cleaned_data = {
    'transactions': df_cleaned.to_dict('records'),
    'customers': df_customers.to_dict('records'),
    'cleaning_log': {
        'original_count': len(df_transactions),
        'cleaned_count': len(df_cleaned),
        'duplicates_removed': len(df_transactions) - len(df_cleaned),
        'outliers_detected': 'Tự động phát hiện và chuẩn hóa',
        'data_quality_score': f"{(len(df_cleaned)/len(df_transactions)*100):.1f}%"
    },
    'metadata': {
        'cleaned_at': datetime.now().isoformat(),
        'description': 'Dữ liệu giao dịch đã được làm sạch và chuẩn hóa'
    }
}

# Lưu ra file JSON
with open('Dữ liệu đã làm sạch.json', 'w', encoding='utf-8') as f:
    json.dump(cleaned_data, f, ensure_ascii=False, indent=2, default=str)

print("Dữ liệu đã làm sạch đã được xuất ra file 'cleaned_transaction_data.json'")
print(f"Số giao dịch ban đầu: {cleaned_data['cleaning_log']['original_count']}")
print(f"Số giao dịch sau làm sạch: {cleaned_data['cleaning_log']['cleaned_count']}")
print(f"Chất lượng dữ liệu: {cleaned_data['cleaning_log']['data_quality_score']}")

# Hiển thị dòng đầu tiên của dữ liệu đã làm sạch
print("\nDữ liệu mẫu sau khi làm sạch:")
df_cleaned.head(3)

Dữ liệu đã làm sạch đã được xuất ra file 'cleaned_transaction_data.json'
Số giao dịch ban đầu: 1000
Số giao dịch sau làm sạch: 1000
Chất lượng dữ liệu: 100.0%

Dữ liệu mẫu sau khi làm sạch:


,transaction_id,customer_id,transaction_time,transaction_type,channel,amount_vnd,payment_method,status,ip_address,user_agent,promotion_code,session_duration,items_count,date,hour,day_of_week,month,time_period
0,TXN20251022CAMBHW,KH940561,2025-07-24 17:09:39.078466,Mua Víp Game,Website Official,100000,Ví ZaloPay,Thành Công,8.244.247.221,Opera/8.82.(Windows NT 6.2; lo-LA) Presto/2.9....,WELCOME10,1194,5,2025-07-24,17,Thursday,2025-07,Chiều
1,TXN20251022CHU3YC,KH849259,2025-07-24 17:35:02.845311,Đăng Ký Tài Khoản Premium,App Gallery,1960000,Thẻ ATM,Thành Công,23.214.124.4,Mozilla/5.0 (compatible; MSIE 6.0; Windows NT ...,WELCOME10,2516,3,2025-07-24,17,Thursday,2025-07,Chiều
2,TXN20251022CFS7I1,KH882304,2025-07-24 18:28:33.795895,Mua Ứng Dụng,Direct Download,2220000,Ví ZaloPay,Thất Bại,73.20.44.162,Opera/8.54.(X11; Linux x86_64; mg-MG) Presto/2...,WELCOME10,3105,1,2025-07-24,18,Thursday,2025-07,Tối


## Phần 3: Tính toán các chỉ số thô ban đầu

In [43]:
# Tính toán các chỉ số cơ bản
def calculate_basic_metrics(df):
    
    # 1. Chỉ số tổng quan
    total_transactions = len(df)
    successful_transactions = (df['status'] == 'Thành Công').sum()
    failed_transactions = (df['status'] == 'Thất Bại').sum()
    pending_transactions = (df['status'] == 'Đang Xử Lý').sum()
    
    total_revenue = df[df['status'] == 'Thành Công']['amount_vnd'].sum()
    avg_transaction_value = df[df['status'] == 'Thành Công']['amount_vnd'].mean()
    
    # 2. Chỉ số theo phân khúc khách hàng
    df_with_segment = df.merge(df_customers[['customer_id', 'segment']], 
                             on='customer_id', how='left')
    
    segment_metrics = {}
    for segment in df_with_segment['segment'].unique():
        segment_data = df_with_segment[df_with_segment['segment'] == segment]
        successful_segment = segment_data[segment_data['status'] == 'Thành Công']
        
        segment_metrics[segment] = {
            'total_transactions': len(segment_data),
            'successful_transactions': len(successful_segment),
            'total_revenue': successful_segment['amount_vnd'].sum(),
            'avg_transaction_value': successful_segment['amount_vnd'].mean() if len(successful_segment) > 0 else 0,
            'success_rate': len(successful_segment) / len(segment_data) * 100
        }
    
    # 3. Chỉ số theo kênh tải
    channel_metrics = {}
    for channel in df['channel'].unique():
        channel_data = df[df['channel'] == channel]
        successful_channel = channel_data[channel_data['status'] == 'Thành Công']
        
        channel_metrics[channel] = {
            'download_count': len(channel_data),
            'successful_downloads': len(successful_channel),
            'total_revenue': successful_channel['amount_vnd'].sum(),
            'avg_revenue_per_download': successful_channel['amount_vnd'].mean() if len(successful_channel) > 0 else 0
        }
    
    # 4. Chỉ số theo thời gian
    df['date'] = pd.to_datetime(df['date'])
    daily_metrics = df.groupby(df['date'].dt.date).agg({
        'transaction_id': 'count',
        'amount_vnd': lambda x: x[df.loc[x.index, 'status'] == 'Thành Công'].sum()
    }).rename(columns={
        'transaction_id': 'transaction_count',
        'amount_vnd': 'daily_revenue'
    })
    
    hourly_metrics = df.groupby('hour').agg({
        'transaction_id': 'count',
        'amount_vnd': lambda x: x[df.loc[x.index, 'status'] == 'Thành Công'].sum()
    }).rename(columns={
        'transaction_id': 'transaction_count',
        'amount_vnd': 'hourly_revenue'
    })
    
    # 5. Chỉ số theo phương thức thanh toán
    payment_metrics = {}
    for payment_method in df['payment_method'].unique():
        payment_data = df[df['payment_method'] == payment_method]
        successful_payment = payment_data[payment_data['status'] == 'Thành Công']
        
        payment_metrics[payment_method] = {
            'usage_count': len(payment_data),
            'successful_count': len(successful_payment),
            'total_amount': successful_payment['amount_vnd'].sum(),
            'success_rate': len(successful_payment) / len(payment_data) * 100
        }
    
    return {
        'overview': {
            'total_transactions': int(total_transactions),
            'successful_transactions': int(successful_transactions),
            'failed_transactions': int(failed_transactions),
            'pending_transactions': int(pending_transactions),
            'success_rate': float(successful_transactions / total_transactions * 100),
            'total_revenue_vnd': int(total_revenue),
            'avg_transaction_value_vnd': float(avg_transaction_value)
        },
        'by_segment': segment_metrics,
        'by_channel': channel_metrics, 
        'by_payment_method': payment_metrics,
        'daily_trends': daily_metrics.to_dict(),
        'hourly_trends': hourly_metrics.to_dict()
    }

# Tính toán chỉ số
print("Đang tính toán các chỉ số...")
metrics = calculate_basic_metrics(df_cleaned)
print("Hoàn thành tính toán chỉ số!")

Đang tính toán các chỉ số...
Hoàn thành tính toán chỉ số!


In [44]:
# Hiển thị tổng quan
overview = metrics['overview']
print("="*60)
print("TỔNG QUAN CHỈ SỐ GIAO DỊCH")
print("="*60)
print(f"Tổng số giao dịch: {overview['total_transactions']:,}")
print(f"Giao dịch thành công: {overview['successful_transactions']:,}")
print(f"Giao dịch thất bại: {overview['failed_transactions']:,}")
print(f"Tỷ lệ thành công: {overview['success_rate']:.2f}%")
print(f"Tổng doanh thu: {overview['total_revenue_vnd']:,.0f} VNĐ")
print(f"Giá trị trung bình/giao dịch: {overview['avg_transaction_value_vnd']:,.0f} VNĐ")

print("\n" + "="*40)
print("CHỈ SỐ THEO PHÂN KHÚC KHÁCH HÀNG")
print("="*40)
for segment, data in metrics['by_segment'].items():
    print(f"\n{segment}:")
    print(f"  - Giao dịch: {data['total_transactions']} | Thành công: {data['successful_transactions']}")
    print(f"  - Doanh thu: {data['total_revenue']:,.0f} VNĐ")
    print(f"  - TB/GD: {data['avg_transaction_value']:,.0f} VNĐ | Tỷ lệ: {data['success_rate']:.1f}%")

print("\n" + "="*40)
print("TOP 5 KÊNH TẢI (Theo doanh thu)")
print("="*40)
top_channels = sorted(metrics['by_channel'].items(), 
                     key=lambda x: x[1]['total_revenue'], reverse=True)[:5]
for channel, data in top_channels:
    print(f"{channel}: {data['total_revenue']:,.0f} VNĐ ({data['download_count']} lượt)")

TỔNG QUAN CHỈ SỐ GIAO DỊCH
Tổng số giao dịch: 1,000
Giao dịch thành công: 597
Giao dịch thất bại: 209
Tỷ lệ thành công: 59.70%
Tổng doanh thu: 301,739,000 VNĐ
Giá trị trung bình/giao dịch: 505,425 VNĐ

CHỈ SỐ THEO PHÂN KHÚC KHÁCH HÀNG

Tiem_Nang:
  - Giao dịch: 227 | Thành công: 122
  - Doanh thu: 17,070,000 VNĐ
  - TB/GD: 139,918 VNĐ | Tỷ lệ: 53.7%

Doanh_Nghiep:
  - Giao dịch: 132 | Thành công: 81
  - Doanh thu: 161,760,000 VNĐ
  - TB/GD: 1,997,037 VNĐ | Tỷ lệ: 61.4%

Tân_Binh:
  - Giao dịch: 275 | Thành công: 171
  - Doanh thu: 7,880,000 VNĐ
  - TB/GD: 46,082 VNĐ | Tỷ lệ: 62.2%

VIP:
  - Giao dịch: 131 | Thành công: 83
  - Doanh thu: 67,120,000 VNĐ
  - TB/GD: 808,675 VNĐ | Tỷ lệ: 63.4%

Thuong_Thien:
  - Giao dịch: 235 | Thành công: 140
  - Doanh thu: 47,909,000 VNĐ
  - TB/GD: 342,207 VNĐ | Tỷ lệ: 59.6%

TOP 5 KÊNH TẢI (Theo doanh thu)
Google Play: 38,660,000 VNĐ (99 lượt)
VNG Game: 35,400,000 VNĐ (122 lượt)
App Store: 34,220,000 VNĐ (92 lượt)
CH Play: 33,430,000 VNĐ (96 lượt)
Websi

In [ ]:
# Xuất metrics ra JSON
def convert_numpy_types(obj):
    if isinstance(obj, np.integer):
        return int(obj)
    elif isinstance(obj, np.floating):
        return float(obj)
    elif isinstance(obj, np.ndarray):
        return obj.tolist()
    elif isinstance(obj, dict):
        # Đảm bảo khóa là chuỗi để JSON ghi được
        return {str(key): convert_numpy_types(value) for key, value in obj.items()}
    elif isinstance(obj, list):
        return [convert_numpy_types(item) for item in obj]
    return obj

# Chuyển đổi các kiểu dữ liệu NumPy và khóa dict không phải chuỗi
metrics_serializable = convert_numpy_types(metrics)

# Tạo dữ liệu metrics với metadata
metrics_data = {
    'metrics': metrics_serializable,
    'metadata': {
        'calculated_at': datetime.now().isoformat(),
        'data_source': 'cleaned_transaction_data.json',
        'total_records_analyzed': len(df_cleaned),
        'analysis_period': {
            'start_date': df_cleaned['transaction_time'].min().isoformat(),
            'end_date': df_cleaned['transaction_time'].max().isoformat()
        },
        'description': 'Các chỉ số phân tích cơ bản từ dữ liệu giao dịch Việt Nam'
    },
    'summary': {
        'key_insights': [
            f"Tổng doanh thu: {overview['total_revenue_vnd']:,.0f} VNĐ",
            f"Tỷ lệ thành công: {overview['success_rate']:.2f}%",
            f"Phân khúc khách hàng hiệu quả nhất: {max(metrics['by_segment'].items(), key=lambda x: x[1]['total_revenue'])[0]}",
            f"Kênh tải doanh thu cao nhất: {max(metrics['by_channel'].items(), key=lambda x: x[1]['total_revenue'])[0]}"
        ]
    }
}

# Lưu ra file JSON
with open('Dữ liệu đã được phân tích.json', 'w', encoding='utf-8') as f:
    json.dump(metrics_data, f, ensure_ascii=False, indent=2, default=str)

print("Chỉ số phân tích đã được xuất ra file 'transaction_metrics.json'")
print(f"Tổng bản ghi phân tích: {metrics_data['metadata']['total_records_analyzed']} giao dịch")
print(f"Khoảng thời gian: {metrics_data['metadata']['analysis_period']['start_date'][:10]} đến {metrics_data['metadata']['analysis_period']['end_date'][:10]}")

print("\nCác điểm chính:")
for insight in metrics_data['summary']['key_insights']:
    print(f"• {insight}")

print("\nCác file đã được tạo:")
print("1. raw_transaction_data.json - Dữ liệu gốc (1000 giao dịch)")
print("2. cleaned_transaction_data.json - Dữ liệu đã làm sạch")
print("3. transaction_metrics.json - Chỉ số phân tích chi tiết")

Chỉ số phân tích đã được xuất ra file 'transaction_metrics.json'
Tổng bản ghi phân tích: 1000 giao dịch
Khoảng thời gian: 2025-07-24 đến 2025-10-22

Các điểm chính:
• Tổng doanh thu: 301,739,000 VNĐ
• Tỷ lệ thành công: 59.70%
• Phân khúc khách hàng hiệu quả nhất: Doanh_Nghiep
• Kênh tải doanh thu cao nhất: Google Play

Các file đã được tạo:
1. raw_transaction_data.json - Dữ liệu gốc (1000 giao dịch)
2. cleaned_transaction_data.json - Dữ liệu đã làm sạch
3. transaction_metrics.json - Chỉ số phân tích chi tiết
